<a href="https://colab.research.google.com/github/akmilton11/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-05 23:56:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.70MB/s    in 0.6s    

2021-08-05 23:56:59 (1.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
Review_df = df
Review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = Review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [12]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

total_votes_df = vine_df.filter(col("total_votes")  >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|     

In [37]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

helpful_votes_calc_df = total_votes_df.withColumn('percentage_helpful', total_votes_df['helpful_votes']/total_votes_df["total_votes"])
helpful_votes_df = helpful_votes_calc_df.filter(col("percentage_helpful")  > .5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|0.8518518518518519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|               1.0|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|              0.95|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|0.7283950617283951|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|0.9565217391304348|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|               1.0|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|               1.0|
|R29IYHPYD14AGI|          3|  

In [38]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_paid_df = helpful_votes_df.filter(col("vine")  == 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|              0.95|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|0.9565217391304348|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|               1.0|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|0.9259259259259259|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|             0.875|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|0.8928571428571429|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|0.9090909090909091|
|R2H4VEFWHP50M3|          5|  

In [39]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_unpaid_df = helpful_votes_df.filter(col("vine")  == 'N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|0.8518518518518519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|               1.0|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|0.7283950617283951|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|               1.0|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|               1.0|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|               1.0|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|               1.0|
|R2GHOWIWDVLN18|          1|  

In [63]:
# dataframe.select('NAME').where(dataframe.ID>5).count()
total_unpaid_ratings = vine_unpaid_df.select('star_rating').count()
print(total_unpaid_ratings)

61849


In [64]:
# dataframe.select('NAME').where(dataframe.ID>5).count()
total_unpaid_5_ratings = vine_unpaid_df.select('vine').where(vine_unpaid_df.star_rating==5).count()
print(total_unpaid_5_ratings)

29950


In [65]:
# dataframe.select('NAME').where(dataframe.ID>5).count()
total_paid_ratings = vine_paid_df.select('star_rating').count()
print(total_paid_ratings)

1266


In [66]:
# dataframe.select('NAME').where(dataframe.ID>5).count()
total_paid_5_ratings = vine_paid_df.select('vine').where(vine_paid_df.star_rating==5).count()
print(total_paid_5_ratings)

432


In [67]:
percentage_unpaid_5_ratings = total_unpaid_5_ratings / total_unpaid_ratings
print(percentage_unpaid_5_ratings)

0.4842438842988569


In [68]:
percentage_paid_5_ratings = total_paid_5_ratings / total_paid_ratings
print(percentage_paid_5_ratings)

0.3412322274881517
